### 1. 성분명 정규화

In [4]:
import re
import pandas as pd

def split_and_normalize_ingredients_v3(ingredients_str: str):
    if pd.isna(ingredients_str):
        return []

    s = str(ingredients_str).strip()

    # 0) "성분:" 같은 prefix가 있으면 그 뒤만 사용 (설명문 제거)
    #    예: "성분: XXX 정제수, ..." -> "정제수, ..."
    m = re.search(r"(성분\s*[:：]\s*)(.*)", s)
    if m:
        s = m.group(2).strip()

    # 1) () [] 안 내용 제거
    s = re.sub(r"\([^)]*\)", "", s)
    s = re.sub(r"\[[^\]]*\]", "", s)

    # 2) 1,2- 패턴 보호 (쉼표 split에 찢어지지 않게)
    s = re.sub(r'(\d)\s*,\s*(\d)\s*-', r'\1§\2-', s)

    # 3) 다양한 구분자 통일: @ ; | 를 쉼표로
    s = s.replace("@", ",").replace(";", ",").replace("|", ",")

    # 4) 쉼표 기준 분리
    parts = [p.strip() for p in s.split(",")]

    tokens = []
    for p in parts:
        if not p:
            continue

        # 5) 보호 문자 복구
        p = p.replace("§", ",")

        # 6) 특수점 통일
        p = p.replace("·", " ").replace("ㆍ", " ")

        # 7) 앞/뒤 특수기호 제거 (', +, *, 등)
        p = re.sub(r"^[\*\+\#★'\"·\s]+", "", p)
        p = re.sub(r"[\*\+\#★'\"·\s]+$", "", p)

        # 8) 끝에 붙는 "*100", "*1", "**100" 같은 꼬리표 제거
        #    예: "소듐하이드록사이드*100" -> "소듐하이드록사이드"
        p = re.sub(r"[\*\+]+?\s*\d+\s*$", "", p).strip()

        # 9) 공백 정리
        p = re.sub(r"\s+", " ", p).strip()

        # 10) 제품명+성분이 붙은 케이스 완화:
        #     토큰 안에 "정제수" 같은 대표 성분이 포함돼 있으면 그 성분부터 잘라냄
        #     (원하면 대표 성분 리스트를 더 늘릴 수 있음)
        for anchor in ["정제수", "글리세린", "부틸렌글라이콜", "1,2-헥산다이올"]:
            idx = p.find(anchor)
            if idx > 0:
                p = p[idx:].strip()
                break

        if p:
            tokens.append(p)

    return tokens


In [7]:
import pandas as pd

df = pd.read_csv("/Users/jeong-yeon-u/Desktop/GNN-based-Oliveyoung-Cosmetic-Recommendation-System-/products_mist_oil.csv")

# 컬럼명 변경
df = df.rename(columns={"ingredient": "ingredients"})

# 저장(원본 덮어쓰기)
df.to_csv("/Users/jeong-yeon-u/Desktop/GNN-based-Oliveyoung-Cosmetic-Recommendation-System-/products_mist_oil.csv", index=False)

# 확인
df.head()



,product_id,category,brand,product_name,ingredients,product_rating
0,M40,미스트/오일,바이오힐보,판테셀 리페어시카 앰플미스트,"정제수, 네오펜틸글라이콜다이헵타노에이트, 1,2-헥산다이올, 펜틸렌글라이콜, 소듐클...",4.6
1,M29,미스트/오일,눅스,윌 프로디쥬스 멀티 드라이 오일,"코코-카프릴레이트/카프레이트, 퀸즈랜드넛오일, 다이카프릴릴에터, 카프릴릭/카프릭트라...",4.8
2,M30,미스트/오일,로벡틴,인텐스 글로우 오일,"카프릴릭/카프릭트라이글리세라이드, 해바라기씨오일, 동백나무씨오일, 스쿠알란, 올리브...",4.7
3,M36,미스트/오일,라로슈포제,세로징크 세범 컨트롤 토닝 미스트,"정제수, 소듐클로라이드, 징크설페이트",4.7
4,M37,미스트/오일,눅스,윌 프로디쥬스 멀티 플로럴 오일,"코코-카프릴레이트/카프레이트, 퀸즈랜드넛오일, 다이카프릴릴에터, 카프릴릭/카프릭트라...",4.8


In [8]:
import re
import pandas as pd

# ===== 경로 설정 =====
input_path = "/Users/jeong-yeon-u/Desktop/GNN-based-Oliveyoung-Cosmetic-Recommendation-System-/products_mist_oil.csv"     # 원본 파일
output_path = "mistoil_ingredients_cleaned.csv"  # 결과 파일

# ===== 데이터 로드 =====
df = pd.read_csv(input_path)

# ===== ingredients 컬럼 전처리 =====
df["ingredients_clean_list"] = df["ingredients"].apply(split_and_normalize_ingredients_v3)

# 문자열 형태도 하나 더 만들어두면 편함
df["ingredients_clean_str"] = df["ingredients_clean_list"].apply(lambda x: "|".join(x))

# ===== 저장 =====
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print("완료!")
print(df[["ingredients", "ingredients_clean_str"]].head())


완료!
                                         ingredients  \
0  정제수, 네오펜틸글라이콜다이헵타노에이트, 1,2-헥산다이올, 펜틸렌글라이콜, 소듐클...   
1  코코-카프릴레이트/카프레이트, 퀸즈랜드넛오일, 다이카프릴릴에터, 카프릴릭/카프릭트라...   
2  카프릴릭/카프릭트라이글리세라이드, 해바라기씨오일, 동백나무씨오일, 스쿠알란, 올리브...   
3                               정제수, 소듐클로라이드, 징크설페이트   
4  코코-카프릴레이트/카프레이트, 퀸즈랜드넛오일, 다이카프릴릴에터, 카프릴릭/카프릭트라...   

                               ingredients_clean_str  
0  정제수|네오펜틸글라이콜다이헵타노에이트|1,2-헥산다이올|펜틸렌글라이콜|소듐클로라이드...  
1  코코-카프릴레이트/카프레이트|퀸즈랜드넛오일|다이카프릴릴에터|카프릴릭/카프릭트라이글리...  
2  카프릴릭/카프릭트라이글리세라이드|해바라기씨오일|동백나무씨오일|스쿠알란|올리브오일|알...  
3                                 정제수|소듐클로라이드|징크설페이트  
4  코코-카프릴레이트/카프레이트|퀸즈랜드넛오일|다이카프릴릴에터|카프릴릭/카프릭트라이글리...  


### 2. 정규화된 성분명 unique value 리스트로 만들기 (count 셀겸..)

In [9]:
from collections import defaultdict
import pandas as pd

N = 500

# ingredient -> set(product_idx)
ingredient_product_set = defaultdict(set)

# ingredient -> category -> set(product_idx)
ingredient_category_product_set = defaultdict(lambda: defaultdict(set))

# 집계
for idx, row in df.iterrows():
    category = row["category"]
    for ing in row["ingredients_clean_list"]:
        ingredient_product_set[ing].add(idx)
        ingredient_category_product_set[ing][category].add(idx)

# 전체 제품 수
ingredient_total_counts = {
    ing: len(products)
    for ing, products in ingredient_product_set.items()
}

# 전체 카테고리 목록
all_categories = sorted(df["category"].dropna().unique())

# DataFrame 생성
rows = []

for ing in ingredient_product_set.keys():
    row = {
        "ingredient": ing,
        "product_count": ingredient_total_counts[ing],
    }

    for cat in all_categories:
        row[f"cat_{cat}"] = len(ingredient_category_product_set[ing].get(cat, set()))

    rows.append(row)

count_df = pd.DataFrame(rows).sort_values("product_count", ascending=False)

# 비율 컬럼 추가
for cat in all_categories:
    count_df[f"ratio_{cat}"] = (
        count_df[f"cat_{cat}"] / count_df["product_count"]
    )

# Top-N
top_ingredients = count_df.head(N)["ingredient"].tolist()


In [10]:
count_df.to_csv("mistoil_ingredients_count.csv", index=False, encoding="utf-8-sig")

### 3. 위에 저장된 리스트 안의 성분명을 하나씩 대한성분협회에 검색 (검색되지 않은 성분을 따로 저장)

In [6]:
# 이 셀을 실행하여 필요한 라이브러리를 설치합니다.
!pip install pandas selenium webdriver-manager openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 2.5 MB/s  0:00:03m0:00:0100:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
  Attempting uninstall: certifi━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  2/10 [urllib3]
    Found existing installation: certifi 2025.8.3━━━━━━━━━━━━━  2/10 [urllib3]
    Uninstalling certifi-2025.8.3:━━━━━━━━━━━━━━━━━━━━━━━━━━━━  2/10 [urllib3]
      Successfully uninstalled certifi-2025.8.3━━━━━━━━━━━━━━━  2/10 [urllib3]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [selenium]/10 [selenium]


In [11]:
import pandas as pd
import time
import random
import os
import sys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ==========================================
# 1. 파일 및 경로 설정
# ==========================================
# Mac의 다운로드 폴더를 기본 경로로 잡습니다.
#base_path = os.path.expanduser('/Users/jeong-yeon-u/Desktop/GNN-based-Oliveyoung-Cosmetic-Recommendation-System- ') 

# 파일 이름 설정 (사용자님 파일명)
FILE_NAME = 'mistoil_ingredients_count.csv'
TARGET_COLUMN = 'ingredient'

# 전체 경로 결합
file_path = os.path.join(FILE_NAME)

print(f"📂 '{file_path}' 데이터를 읽는 중...")

# ==========================================
# 2. 데이터 불러오기
# ==========================================
if not os.path.exists(file_path):
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
    print("다운로드 폴더에 파일이 있는지 확인해주세요.")
    # 노트북에서는 exit() 대신 에러를 일으켜 멈추는 게 좋습니다.
    raise FileNotFoundError("파일 없음")

df = pd.read_csv(file_path)

# 검색할 리스트 만들기
ingredients_set = set()
for item in df[TARGET_COLUMN]:
    if pd.isna(item):
        continue
    clean_name = str(item).strip()
    if clean_name:
        ingredients_set.add(clean_name)

search_list = sorted(list(ingredients_set))
print(f"✅ 총 {len(search_list)}개의 성분을 검증합니다.")

# ==========================================
# 3. 크롬 브라우저 설정
# ==========================================
options = webdriver.ChromeOptions()
options.add_argument('--window-size=1920,1080')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--remote-allow-origins=*") # Mac 보안 설정

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# ==========================================
# 4. 검색 및 검증 루프
# ==========================================
unknown_ingredients = []   
verified_ingredients = []  

try:
    # 사이트 접속
    url = "https://kcia.or.kr/cid/main/"
    driver.get(url)
    print("🌏 사이트 접속 완료. 3초 후 검색을 시작합니다...")
    time.sleep(3) 

    for index, ing_name in enumerate(search_list):
        # 진행 상황 출력 (end='\r'을 쓰면 한 줄에서 숫자가 바뀝니다)
        print(f"[{index+1}/{len(search_list)}] 검색 중: {ing_name[:15]}... ", end="")
        
        try:
            # (1) 검색창 찾기
            try:
                search_box = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='text']"))
                )
            except:
                search_box = driver.find_element(By.NAME, "keyword")
            
            # (2) 검색어 입력
            search_box.clear()
            search_box.send_keys(ing_name)
            search_box.send_keys(Keys.RETURN)
            
            time.sleep(1.0) # 로딩 대기

            # (3) 결과 정밀 검증
            page_source = driver.page_source
            
            # 3-1. 결과 없음 체크
            if "검색된 결과가 없습니다" in page_source or "데이터가 존재하지 않습니다" in page_source:
                print("-> ❌ 없음 (미등록)")
                unknown_ingredients.append(ing_name)
                
            else:
                # 3-2. 정밀 비교
                try:
                    first_row = driver.find_element(By.CSS_SELECTOR, "tbody tr:first-child")
                    result_text = first_row.text
                    
                    search_key_clean = ing_name.replace(" ", "").replace(",", "").replace("|", "")
                    result_text_clean = result_text.replace(" ", "")

                    if len(search_key_clean) > len(result_text_clean): 
                        print("-> ❌ 불일치 (부분 매칭)")
                        unknown_ingredients.append(ing_name)
                    elif ing_name in result_text:
                        print("-> ⭕️ 확인됨")
                        verified_ingredients.append(ing_name)
                    else:
                        print("-> ❌ 이름 다름")
                        unknown_ingredients.append(ing_name)
                        
                except Exception as e:
                    print(f"-> ⚠️ 검증 실패")
                    unknown_ingredients.append(ing_name)

            time.sleep(random.uniform(0.5, 1.0))

        except Exception as e:
            print(f"-> ⚠️ 에러 발생: {e}")
            unknown_ingredients.append(ing_name)

except KeyboardInterrupt:
    print("\n⏹ 사용자에 의해 중단되었습니다.")

finally:
    # 에러가 나거나 중단해도 브라우저는 끄도록 함
    driver.quit()
    print("\n브라우저를 종료했습니다.")

# ==========================================
# 5. 결과 저장
# ==========================================
print("\n" + "="*30)
print(f"🎉 검증 완료!")
print(f"검색 성공: {len(verified_ingredients)}개")
print(f"검색 실패/불일치: {len(unknown_ingredients)}개")

if unknown_ingredients:
    output_path = os.path.join("unknown_ingredients.xlsx")
    df_result = pd.DataFrame(unknown_ingredients, columns=['Need_Normalization'])
    df_result.to_excel(output_path, index=False)
    print(f"📂 결과 파일 저장 완료: {output_path}")
else:
    print("✨ 모든 성분이 정상적으로 확인되었습니다.")

📂 'mistoil_ingredients_count.csv' 데이터를 읽는 중...
✅ 총 551개의 성분을 검증합니다.
🌏 사이트 접속 완료. 3초 후 검색을 시작합니다...
[1/551] 검색 중: 1,2-헥산다이올... -> ⭕️ 확인됨
[2/551] 검색 중: 1,2-헥산디올... -> ⭕️ 확인됨
[3/551] 검색 중: 2,3-부탄다이올... -> ⭕️ 확인됨
[4/551] 검색 중: C13-15알케인... -> ⭕️ 확인됨
[5/551] 검색 중: 가지열매추출물... -> ⭕️ 확인됨
[6/551] 검색 중: 갈바눔수지오일... -> ⭕️ 확인됨
[7/551] 검색 중: 감추출물... -> ⭕️ 확인됨
[8/551] 검색 중: 개다래추출물... -> ⭕️ 확인됨
[9/551] 검색 중: 개똥쑥추출물... -> ⭕️ 확인됨
[10/551] 검색 중: 검은깨추출물... -> ⭕️ 확인됨
[11/551] 검색 중: 겔리듐 카르틸라기네움추출물... -> ⭕️ 확인됨
[12/551] 검색 중: 결명씨추출물... -> ⭕️ 확인됨
[13/551] 검색 중: 고구마뿌리추출물... -> ⭕️ 확인됨
[14/551] 검색 중: 과라나씨추출물... -> ⭕️ 확인됨
[15/551] 검색 중: 구기자추출물... -> ⭕️ 확인됨
[16/551] 검색 중: 귀리커넬추출물... -> ⭕️ 확인됨
[17/551] 검색 중: 귤껍질추출물... -> ⭕️ 확인됨
[18/551] 검색 중: 글라이코실트레할로오스... -> ⭕️ 확인됨
[19/551] 검색 중: 글루코노락톤... -> ⭕️ 확인됨
[20/551] 검색 중: 글루코놀락톤... -> ⚠️ 검증 실패
[21/551] 검색 중: 글루코오스... -> ⭕️ 확인됨
[22/551] 검색 중: 글루타티온... -> ⭕️ 확인됨
[23/551] 검색 중: 글리세레스-26... -> ⭕️ 확인됨
[24/551] 검색 중: 글리세린... -> ⭕️ 확인됨
[25/551] 검색 중: 글리세린-3... -> ⭕️ 확인됨
[26/551

### 4. 맵핑 테이블 작성 (우리가 직접..)
여기서 만들어진 파일의 'Need_Normalization' 컬럼 옆에 'nomalized' 컬럼 만들어서 저희가 직접 고쳐야해요ㅜㅜ 저는 일부 단어 검색해보면서 맞는 걸로 고쳐뒀습니당..

### 5. 맵핑 데이터 대체 
원래 성분명이 존재하는 테이블에서 'Need_Normalization' 컬럼 값과 일치하는 성분들을 'nomalized'컬럼값과 맵핑하여 대체

In [13]:
import pandas as pd
import ast # 문자열로 된 리스트("['a', 'b']")를 진짜 리스트(['a', 'b'])로 바꿔주는 도구
import os

# ==========================================
# 1. 파일 경로 설정 (Mac 다운로드 폴더 자동 지정)
# ==========================================
# 사용자 홈 디렉토리의 다운로드 폴더 경로
#base_path = os.path.expanduser('~/Downloads')

# 파일 이름 (사용자님 파일명 그대로)
csv_file_name = 'mistoil_ingredients_cleaned.csv'
excel_file_name = 'unknown_ingredients.xlsx'

csv_path = 'mistoil_ingredients_cleaned.csv'
excel_path = 'unknown_ingredients.xlsx'

print("📂 파일 로딩 중...")

# ==========================================
# 2. 데이터 불러오기
# ==========================================
try:
    # (1) 성분 데이터 (원본)
    df_products = pd.read_csv(csv_path)
    
    # (2) 정규화 규칙 데이터 (엑셀)
    df_mapping = pd.read_excel(excel_path)
    
    print("✅ 파일 읽기 성공!")

except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없습니다: {e}")
    exit()

# ==========================================
# 3. 교체 규칙 사전(Dictionary) 만들기
# ==========================================
# 엑셀 파일에서 매핑 규칙 생성
# Key: Need_Normalization (바꿔야 할 옛날 이름)
# Value: normalized (바뀔 새 이름)
# 엑셀의 NaN(빈값)을 방지하기 위해 dropna() 혹은 fillna() 처리 필요할 수 있음
df_mapping = df_mapping.dropna(subset=['Need_Normalization', 'normalized'])

# { '옛날이름1': '새이름1', '옛날이름2': '새이름2', ... } 형태의 사전 생성
replace_dict = dict(zip(df_mapping['Need_Normalization'], df_mapping['normalized']))

print(f"🛠 총 {len(replace_dict)}개의 성분 교체 규칙을 적용합니다.")

# ==========================================
# 4. 리스트 순회하며 성분명 바꾸기
# ==========================================

def normalize_ingredient_list(row_val):
    """
    한 행의 리스트 문자열을 받아서, 
    사전에 있는 성분이면 이름을 바꾸고, 없으면 그대로 둠.
    """
    if pd.isna(row_val):
        return []
    
    try:
        # 1. CSV에는 리스트가 문자열 "['성분1', '성분2']"로 저장되어 있으므로
        #    이걸 진짜 파이썬 리스트 ['성분1', '성분2']로 변환
        current_list = ast.literal_eval(str(row_val))
        
        normalized_list = []
        for ing in current_list:
            # 2. 사전에서 찾기 (get(찾을값, 없으면_그대로_사용))
            # 공백 제거(strip) 후 비교하는 것이 안전함
            ing_clean = ing.strip()
            
            # 교체 대상이면 새 이름으로, 아니면 원래 이름 그대로
            new_name = replace_dict.get(ing_clean, ing_clean)
            normalized_list.append(new_name)
            
        return normalized_list
    
    except (ValueError, SyntaxError):
        # 리스트 형태가 깨져있거나 파싱 불가능한 경우 원본 반환 혹은 빈 리스트
        return row_val

# 적용하기 (오래 걸릴 수 있으니 진행상황 출력)
print("🔄 성분명 변경 작업 시작...")
df_products['ingredients_normalized_list'] = df_products['ingredients_clean_list'].apply(normalize_ingredient_list)

# ==========================================
# 5. 결과 저장
# ==========================================
output_path = 'products_ingredients_final_mistoil.csv'

# 리스트 컬럼을 다시 저장할 때 보기 좋게 하기 위해 (옵션)
# df_products.to_csv(output_path, index=False, encoding='utf-8-sig') 
df_products.to_csv(output_path, index=False) 

print("\n" + "="*30)
print("🎉 작업 완료!")
print(f"📂 저장된 파일 위치: {output_path}")
print("새로 생긴 'ingredients_normalized_list' 컬럼을 확인해보세요.")

📂 파일 로딩 중...
✅ 파일 읽기 성공!
🛠 총 14개의 성분 교체 규칙을 적용합니다.
🔄 성분명 변경 작업 시작...

🎉 작업 완료!
📂 저장된 파일 위치: products_ingredients_final_mistoil.csv
새로 생긴 'ingredients_normalized_list' 컬럼을 확인해보세요.


In [15]:
import pandas as pd
import ast
import os
from collections import Counter

# ==========================================
# 1. 파일 경로 설정 (Mac 다운로드 폴더)
# ==========================================
#base_path = os.path.expanduser('~/Downloads')
input_file_name = 'products_ingredients_final_essence.csv' # 방금 만든 파일
input_path = 'products_ingredients_final_essence.csv'

print(f"📂 '{input_file_name}' 파일을 읽는 중...")

# ==========================================
# 2. 데이터 불러오기 및 카운팅
# ==========================================
try:
    df = pd.read_csv(input_path)
except FileNotFoundError:
    print("❌ 파일을 찾을 수 없습니다. 직전 단계(정규화)를 먼저 실행했는지 확인해주세요.")
    exit()

# 성분 개수를 셀 카운터(Counter) 객체 생성
ingredient_counter = Counter()

print("🔢 성분 카운팅 시작...")

# 'ingredients_normalized_list' 컬럼을 한 줄씩 읽어서 카운트
for item in df['ingredients_normalized_list']:
    if pd.isna(item):
        continue

    try:
        # 문자열로 된 리스트 "['성분A', '성분B']"를 진짜 리스트로 변환
        ing_list = ast.literal_eval(str(item))
        
        # 리스트 안의 성분들을 카운터에 업데이트 (한 번에 다 더해줌)
        ingredient_counter.update(ing_list)
        
    except (ValueError, SyntaxError):
        continue

# ==========================================
# 3. 결과 정리 및 저장
# ==========================================
# 카운터 결과를 데이터프레임으로 변환
# columns: [성분명, 등장횟수]
count_df = pd.DataFrame(ingredient_counter.items(), columns=['ingredient', 'count'])

# 등장 횟수(count) 기준으로 내림차순 정렬 (많이 나온 게 위로)
count_df = count_df.sort_values(by='count', ascending=False)

# 결과 저장
output_file_name = 'ingredient_counts_new.csv'
output_path = os.path.join(output_file_name)

count_df.to_csv(output_path, index=False)

print("\n" + "="*30)
print("🎉 카운팅 완료!")
print(f"총 {len(count_df)}개의 고유 성분이 집계되었습니다.")
print(f"📂 결과 파일: {output_path}")

# 상위 10개 성분 미리보기
print("\n[🔥 가장 많이 쓰인 성분 TOP 10]")
print(count_df.head(10).to_string(index=False))

📂 'products_ingredients_final_essence.csv' 파일을 읽는 중...
🔢 성분 카운팅 시작...

🎉 카운팅 완료!
총 1400개의 고유 성분이 집계되었습니다.
📂 결과 파일: ingredient_counts_new.csv

[🔥 가장 많이 쓰인 성분 TOP 10]
  ingredient  count
        글리세린    462
         정제수    277
   1,2-헥산다이올    261
     부틸렌글라이콜    248
    나이아신아마이드    190
  소듐하이알루로네이트    177
         판테놀    163
        아데노신    153
하이드로제네이티드레시틴    153
        토코페롤    146
